In [1]:
# 1. 필요한 라이브러리들을 불러옵니다.
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# 2. API 키 설정
# .env 파일에서 API 키를 불러옵니다.
load_dotenv()
print("API Key Loaded!")

# 3. 문서 로드 (Load)
# 'data' 폴더에 있는 모든 .txt, .md 파일을 불러옵니다.
loader = DirectoryLoader('./data', glob="**/*.md")
documents = loader.load()
print(f"Loaded {len(documents)} documents.")

# 4. 문서 분할 (Split)
# 불러온 문서를 AI가 처리하기 좋은 작은 조각(chunk)으로 나눕니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Split into {len(texts)} chunks.")

# 5. 임베딩 및 벡터 저장소 구축 (Embed & Store)
# 각 텍스트 조각을 벡터로 변환(임베딩)하고, 검색할 수 있도록 FAISS 벡터 저장소에 저장합니다.
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)
print("Vector store is ready.")

# 6. 검색 및 답변 생성 (Retrieve & Generate)
# LLM 모델을 설정합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0)

# RAG 체인을 생성합니다.
# 이 체인은 질문이 들어오면 벡터 저장소에서 관련 문서를 찾고(retrieve),
# 찾은 문서와 질문을 함께 LLM에 보내 답변을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

# 7. 질문하기!
query = "프로젝트의 목적이 뭐야?" # data 폴더에 넣은 README.md 내용과 관련된 질문
result = qa_chain.invoke(query)

print("\n--- 질문 ---")
print(query)
print("\n--- 답변 ---")
print(result['result'])

/Users/jungkang/project/ybigta-rag-agent/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


API Key Loaded!


libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Loaded 1 documents.
Split into 5 chunks.
Vector store is ready.

--- 질문 ---
프로젝트의 목적이 뭐야?

--- 답변 ---
프로젝트의 목적은 아마추어 및 동호회 농구 경기에서 발생하는 기록 인력 부족 문제를 해결하고, 선수들에게 데이터 기반의 경기 분석 경험을 제공하는 것입니다. 이를 위해 컴퓨터 비전 기술을 활용하여 경기 기록 과정을 자동화하고, 휴먼 에러를 최소화하며, 실시간으로 경기 상황을 분석하고 스코어보드를 업데이트하는 시스템을 구축하는 것입니다.


In [2]:
pip install "unstructured[md]"

     |████████████████████████████████| 106 kB 1.3 MB/s eta 0:00:01
You should consider upgrading via the '/Users/jungkang/project/ybigta-rag-agent/.venv/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
